In [1]:
import glob
import json
from tqdm.notebook import tqdm
import pprint
import pandas as pd

In [2]:
root = 'e:\\phd\\test_repos\\results\\'

In [3]:
repos = [
    'azure-service-operator',
    'backend',
    'boost',
    'chainlink',
    'delivery-offering',
    'docker-ce',
    'go-ethereum',
    'go-redis',
    'grafana',
    'gvisor',
    'kubernetes',
    'moby',
    'modules',
    'sourcegraph',
    'tidb'
]

In [4]:
# class Stats():
#     def __init__(self, name, args, args_depth, args_cnt, return_, returns_depth):
#         self.name = name
#         self.args = args
#         self.args_depth = args_depth
#         self.args_cnt = args_cnt
#         self.return_ = return_
#         self.returns_depth = returns_depth

In [5]:
# def object_decoder(obj):
#     if 'ArgsCnt' not in obj:
#         return None 
#     return Stats(obj['Name'], obj['Args'], obj['ArgsDepth'], obj['ArgsCnt'], obj['Return'], obj['ReturnsDepth'])

In [6]:
d = {}
i = 0 
for r in tqdm(repos):
    files = glob.glob(root + r + '/**/*.json',recursive = True)
    for fname in files:
        with open(fname, 'r') as file:
            lines = []
            for line in file:
                line = line.strip()
                obj = json.loads(line)
                if 'ArgsCnt' not in obj:
                    continue
                d[i] = obj
                d[i]["Source"] = r
                del d[i]["Empty"]
                i+=1
                #df = df.append(obj,ignore_index=True)

  0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
df = pd.DataFrame.from_dict(d, "index")
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols] 

In [8]:
df['∑ArgsDepth'] = df['ArgsDepth'].apply(sum)
df['∑ReturnsDepth'] = df['ReturnsDepth'].apply(sum)
df['MxArgsDepth'] = df['ArgsDepth'].apply(lambda x: max(x, default=0))
df['MxReturnsDepth'] = df['ReturnsDepth'].apply(lambda x: max(x, default=0))
df['TotalDepth'] = df['∑ArgsDepth']+df['∑ReturnsDepth']
df['AvgDepth'] = df['TotalDepth'] / (df['ArgsCnt']+df['Return'])

In [9]:
df = df.drop_duplicates(subset=['Name', 'AvgDepth', 'ArgsCnt', 'Return'], keep='last')

In [10]:
df = df[[
    'Source', 'Name', 
    'ArgsDepth', 'ArgsCnt', '∑ArgsDepth', 'MxArgsDepth',
    'ReturnsDepth', 'Return', '∑ReturnsDepth', 'MxReturnsDepth',
    'TotalDepth', 'AvgDepth',
    'Receiver', 'Args'
]]

In [12]:
df.sort_values('AvgDepth', ascending=False).head(15)

,Source,Name,ArgsDepth,ArgsCnt,∑ArgsDepth,MxArgsDepth,ReturnsDepth,Return,∑ReturnsDepth,MxReturnsDepth,TotalDepth,AvgDepth,Receiver,Args
585446,sourcegraph,nextHook,[],0,0,0,[5],1,5,5,5,5.0,S3APIDeleteObjectsFunc,[]
585443,sourcegraph,PushHook,[5],1,5,5,[],0,0,0,5,5.0,S3APIDeleteObjectsFunc,[anon_func_title]
585442,sourcegraph,SetDefaultHook,[5],1,5,5,[],0,0,0,5,5.0,S3APIDeleteObjectsFunc,[anon_func_title]
139387,boost,AddProtector,[5],1,5,5,[],0,0,0,5,5.0,NoopGCReferenceProtector,[anon_func_title]
612676,tidb,SetTestHook,[5],1,5,5,[],0,0,0,5,5.0,,[anon_func_title]
179354,docker-ce,SetGetAllFunc,[4],1,4,4,[],0,0,0,4,4.0,FakeStore,[anon_func_title]
490328,moby,DebugRequestMiddleware,[4],1,4,4,[4],1,4,4,8,4.0,,[anon_func_title]
271387,go-ethereum,Inspect,[],0,0,0,[4],1,4,4,4,4.0,TxPoolAPI,[]
618384,tidb,getNewStringPK,[],0,0,0,[4],1,4,4,4,4.0,,[]
618380,tidb,getNewIntPK,[],0,0,0,[4],1,4,4,4,4.0,,[]
